# Predict deaths

In [10]:
import pandas as pd
import seaborn as sns

In [11]:
#import data
url="https://github.com/Themaoyc/MDA/blob/main/Data/deaths_predict.csv?raw=true"
df = pd.read_csv(url)
df.head()

,Year,Country,ISO,Total Deaths,CPI,GDP(million dollars),Population,healthexp,tmax,Dis Mag Value,duration
0,2000,Bulgaria,BGR,7,63.549547,"28,124.88",8170172,94.497452,34.600000,34,31
1,2000,Cyprus,CYP,5,63.549547,"10,594.98",943288,750.516724,34.100000,34,1
2,2000,Greece,GRC,27,63.549547,"141,247.28",10805808,885.410400,33.800000,33,1
3,2000,Turkey,TUR,11,63.549547,"171,494.21",63240196,199.499374,27.600000,27,31
4,2000,Pakistan,PAK,24,63.549547,...,142343583,15.581897,29.235238,29,1


## Data Exporation 

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Year                  116 non-null    int64  
 1   Country               116 non-null    object 
 2   ISO                   116 non-null    object 
 3   Total Deaths          116 non-null    int64  
 4   CPI                   116 non-null    float64
 5   GDP(million dollars)  116 non-null    object 
 6   Population            116 non-null    int64  
 7   healthexp             116 non-null    float64
 8   tmax                  116 non-null    float64
 9   Dis Mag Value         116 non-null    int64  
 10  duration              116 non-null    int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 10.1+ KB


In [8]:
df.describe()

,Year,Total Deaths,CPI,Population,healthexp,tmax,Dis Mag Value,duration
count,116.000000,116.000000,116.000000,1.160000e+02,116.000000,116.000000,116.000000,116.000000
mean,2008.956897,1294.543103,78.397740,2.458414e+08,2353.638281,29.178637,28.784483,19.103448
std,6.230908,5934.045809,10.143401,4.142032e+08,2174.756907,5.708539,5.650404,40.361062
min,2000.000000,1.000000,63.549547,4.516300e+05,10.275986,8.100000,8.000000,-9.000000
25%,2003.000000,15.750000,67.888896,1.709518e+07,146.953505,26.475000,26.000000,1.000000
50%,2007.000000,62.000000,76.518679,6.724893e+07,2202.601807,29.235238,29.000000,7.000000
75%,2015.000000,339.750000,87.469932,1.612337e+08,4215.190674,30.100000,30.000000,28.000000
max,2019.000000,55736.000000,94.349092,1.366418e+09,8342.580078,41.800000,41.000000,377.000000


In [13]:
sns.heatmap(df[['Year', 'Country', 'tavg', 'tmin','tmax','prcp','wspd','heatwave'
                     ,'Total Deaths']].corr(),cmap='RdYlGn',annot=True)

NameError: name 'final' is not defined